<a href="https://colab.research.google.com/github/ochamodev/data_science_at_work_project_final/blob/main/DS_Research_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# lib imports

In [1]:
# library imports

    # utils
import os
import shutil

    # data wrangling and graphs
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colormaps

    # dataset balancing
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler, NearMiss

    # models, metrics and feature selection from sklearn

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression

    # tensorflow for DL
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

    # keras layers and models
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers

# load dataset

In [2]:
'''
Datasest found @
https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
'''

import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
print("Path to dataset files:", path)

# copy to local dir
destination = "/content/dataset"
shutil.copytree(path, destination)

# read dataset
df = pd.read_csv("/content/dataset/creditcard.csv")

100%|██████████| 66.0M/66.0M [00:02<00:00, 30.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mlg-ulb/creditcardfraud/versions/3


# autoencoder original chatio

In [10]:
np.random.seed(203)

data = pd.read_csv("/content/dataset/creditcard.csv")
data["Time"] = data["Time"].apply(lambda x : x / 3600 % 24)

non_fraud = data[data['Class'] == 0].sample(1000)
fraud = data[data['Class'] == 1]

df = pd.concat([non_fraud, fraud]).sample(frac=1).reset_index(drop=True)
X = df.drop(['Class'], axis = 1).values
Y = df["Class"].values

# input layer
input_layer = Input(shape=(X.shape[1],))

# encoding part
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

# decoding part
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

# output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)

# model architecture
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adadelta", loss="mse")

x = data.drop(["Class"], axis=1)
y = data["Class"].values
x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scale[y == 0], x_scale[y == 1]

autoencoder.fit(x_norm[0:2000], x_norm[0:2000],
                batch_size = 256, epochs = 10,
                shuffle = True, validation_split = 0.20);

# Latent Representations
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

# Generate the hidden representations of two classes : non-fraud and fraud by predicting the raw inputs
norm_hid_rep = hidden_representation.predict(x_norm[:3000])
fraud_hid_rep = hidden_representation.predict(x_fraud)

# create a training dataset using the latent representations
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

# Linear Classifier
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 268ms/step - loss: 0.9914 - val_loss: 0.8748
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9908 - val_loss: 0.8742
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9900 - val_loss: 0.8736
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9895 - val_loss: 0.8730
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9889 - val_loss: 0.8724
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9882 - val_loss: 0.8718
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9876 - val_loss: 0.8712
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9867 - val_loss: 0.8706
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9860 - val_loss: 0.8700
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9860 - val_loss: 0.8693
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

Classification Report: 
              precision    recall  f1-score   support

         0.0       

# autoencoder version nosotros

In [15]:
np.random.seed(203)

data = pd.read_csv("/content/dataset/creditcard.csv")
data["Time"] = data["Time"].apply(lambda x : x / 3600 % 24)

non_fraud = data[data['Class'] == 0]
fraud = data[data['Class'] == 1]

df = pd.concat([non_fraud, fraud]).sample(frac=1).reset_index(drop=True)
X = df.drop(['Class'], axis = 1).values
Y = df["Class"].values

# input layer
input_layer = Input(shape=(X.shape[1],))

# encoding part
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

# decoding part
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

# output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)

# model architecture
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adadelta", loss="mse")

x = data.drop(["Class"], axis=1)
y = data["Class"].values
x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scale[y == 0], x_scale[y == 1]

autoencoder.fit(x_norm,
                x_norm,
                batch_size = 256,
                epochs = 10,
                shuffle = True,
                validation_split = 0.20)

Epoch 1/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.9161 - val_loss: 0.8349
Epoch 2/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7839 - val_loss: 0.6934
Epoch 3/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6438 - val_loss: 0.5522
Epoch 4/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.5103 - val_loss: 0.4396
Epoch 5/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.4039 - val_loss: 0.3533
Epoch 6/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3201 - val_loss: 0.2805
Epoch 7/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2553 - val_loss: 0.2305
Epoch 8/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2121 - val_loss: 0.1995
Epoch 9/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1851 - val_loss: 0.1798
Epoch 10/10
889/889 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1678 - val_loss: 0.1673


In [22]:
# Latent Representations
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

## A: MODELO BASE SIN OVER/UNDER SAMPLING

In [23]:
# Generate the hidden representations of two classes : non-fraud and fraud by predicting the raw inputs
norm_hid_rep = hidden_representation.predict(x_norm)
fraud_hid_rep = hidden_representation.predict(x_fraud)

# create a training dataset using the latent representations
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

# Linear Classifier
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

8885/8885 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

Classification Report: 
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     71080
         1.0       0.85      0.37      0.51       122

    accuracy                           1.00     71202
   macro avg       0.92      0.68      0.76     71202
weighted avg       1.00      1.00      1.00     71202


Accuracy Score:  0.998806213308615


## B: MODELO UNDERSAMPLING

In [81]:
#---------------- Random Undersampler -------------------#

# make the large one 5 times the size of the smaller one
fraud_count = x_fraud.shape[0]
desired_norm_count = fraud_count * 5

rus = RandomUnderSampler(sampling_strategy={0:desired_norm_count}, random_state=42)
x_undersampled, y_undersampled = rus.fit_resample(x_scale, y)

In [82]:
x_undersampled = pd.DataFrame(x_undersampled, columns=x.columns)
y_undersampled = pd.DataFrame(y_undersampled, columns=["Class"])

In [83]:
undersampled = pd.concat([x_undersampled, y_undersampled], axis=1)
x_norm_undersampled = undersampled[undersampled["Class"] == 0]
x_norm_undersampled = x_norm_undersampled.drop(["Class"], axis=1)

In [84]:
# Generate the hidden representations of two classes : non-fraud and fraud by predicting the raw inputs
norm_hid_rep = hidden_representation.predict(x_norm_undersampled.values)
fraud_hid_rep = hidden_representation.predict(x_fraud)

# create a training dataset using the latent representations
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

# Linear Classifier
train_x, test_x, train_y, test_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(test_x)

print ("")
print ("Classification Report: ")
print (classification_report(test_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(test_y, pred_y))

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Classification Report: 
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97       626
         1.0       0.98      0.71      0.82       112

    accuracy                           0.95       738
   macro avg       0.96      0.85      0.90       738
weighted avg       0.95      0.95      0.95       738


Accuracy Score:  0.9525745257452575


## E: MODELO NEAR MISS UNDERSAMPLING

In [100]:
#---------------- Near Miss Undersampling -------------------#
fraud_count = x_fraud.shape[0]
desired_norm_count = fraud_count * 5

nm = NearMiss(sampling_strategy={0:desired_norm_count})
x_under_nm, y_under_nm = nm.fit_resample(x_scale, y)
x_under_nm = pd.DataFrame(x_under_nm, columns=x.columns)
y_under_nm = pd.DataFrame(y_under_nm, columns=["Class"])
under_nm = pd.concat([x_under_nm, y_under_nm], axis=1)
x_norm_under_nm = under_nm[under_nm["Class"] == 0]
x_norm_under_nm = x_norm_under_nm.drop(["Class"], axis=1)

In [101]:
# Generate the hidden representations of two classes : non-fraud and fraud by predicting the raw inputs
norm_hid_rep = hidden_representation.predict(x_norm_under_nm.values)
fraud_hid_rep = hidden_representation.predict(x_fraud)

# create a training dataset using the latent representations
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

# Linear Classifier
train_x, test_x, train_y, test_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(test_x)

print ("")
print ("Classification Report: ")
print (classification_report(test_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(test_y, pred_y))

77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Classification Report: 
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97       619
         1.0       0.98      0.71      0.83       119

    accuracy                           0.95       738
   macro avg       0.96      0.86      0.90       738
weighted avg       0.95      0.95      0.95       738


Accuracy Score:  0.9512195121951219


## D: MODELO RANDOM OVERSAMPLING

In [95]:
#---------------- Random Oversampler -------------------#
ros = RandomOverSampler(sampling_strategy=0.25, random_state=42)
x_oversampled, y_oversampled = ros.fit_resample(x_scale, y)
x_oversampled = pd.DataFrame(x_oversampled, columns=x.columns)
y_oversampled = pd.DataFrame(y_oversampled, columns=["Class"])
oversampled = pd.concat([x_oversampled, y_oversampled], axis=1)
x_norm_oversampled = oversampled[oversampled["Class"] == 0]
x_norm_oversampled = x_norm_oversampled.drop(["Class"], axis=1)

In [96]:
# Generate the hidden representations of two classes : non-fraud and fraud by predicting the raw inputs
norm_hid_rep = hidden_representation.predict(x_norm_oversampled.values)
fraud_hid_rep = hidden_representation.predict(x_fraud)

# create a training dataset using the latent representations
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

# Linear Classifier
train_x, test_x, train_y, test_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(test_x)

print ("")
print ("Classification Report: ")
print (classification_report(test_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(test_y, pred_y))

8885/8885 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Classification Report: 
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     71077
         1.0       0.86      0.44      0.58       125

    accuracy                           1.00     71202
   macro avg       0.93      0.72      0.79     71202
weighted avg       1.00      1.00      1.00     71202


Accuracy Score:  0.9988904806044774


## C: MODELO SMOTE OVERSAMPLING

In [102]:
#---------------- Random Oversampler -------------------#
smote = SMOTE(sampling_strategy=0.25, random_state=42)
x_over_smote, y_over_smote = smote.fit_resample(x_scale, y)
x_over_smote = pd.DataFrame(x_over_smote, columns=x.columns)
y_over_smote = pd.DataFrame(y_over_smote, columns=["Class"])
over_smote = pd.concat([x_over_smote, y_over_smote], axis=1)
x_norm_over_smote = over_smote[over_smote["Class"] == 0]
x_norm_over_smote = x_norm_over_smote.drop(["Class"], axis=1)

In [103]:
# Generate the hidden representations of two classes : non-fraud and fraud by predicting the raw inputs
norm_hid_rep = hidden_representation.predict(x_norm_over_smote.values)
fraud_hid_rep = hidden_representation.predict(x_fraud)

# create a training dataset using the latent representations
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

# Linear Classifier
train_x, test_x, train_y, test_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(test_x)

print ("")
print ("Classification Report: ")
print (classification_report(test_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(test_y, pred_y))

8885/8885 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

Classification Report: 
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     71068
         1.0       0.79      0.37      0.51       134

    accuracy                           1.00     71202
   macro avg       0.90      0.69      0.75     71202
weighted avg       1.00      1.00      1.00     71202


Accuracy Score:  0.99863767871689
